# This notebook generates the signal for the FFT problem

The goal is to generate a signal that is hard to analyze - it has noise, it grows with time (with no frequency to the signal), and some components are missing.  It's meant to be an interesting challenge that builds on top of what they did in class.

Some links that were useful while I was doing this are:

https://docs.scipy.org/doc/scipy/tutorial/fft.html

https://realpython.com/python-scipy-fft/#practical-example-remove-unwanted-noise-from-audio

https://dsp.stackexchange.com/questions/72005/calculate-the-magnitude-and-phase-of-a-signal-at-a-particular-frequency-in-pytho

https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fft.html

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.fftpack

In [ ]:
np.random.seed(8675309)

# make this similar to what we did before.
t_start = 0
t_stop = 6.0*np.pi

N_samples = 12000

dt = (t_stop-t_start)/N_samples

t = np.linspace(0,N_samples*dt,N_samples,endpoint=False)

'''
Signal has:  
  * constant offset
  * cosine and sine terms
  * a component that grows steadily over time
  * positive and negative offsets
'''
fx = (0.2*t-np.pi) - 1.5*np.cos(10.0*np.pi*t) \
     + 3.0*np.cos(22.0*np.pi*t)  - 1.0*np.sin(62.0*np.pi*t) + 3.0

'''
Add noise to the signal
'''
fx_noisy = fx + np.random.normal(loc=0,scale=0.5,size=fx.shape)

plt.plot(t,fx_noisy,'r.',alpha=0.5)
plt.plot(t,fx,'b-',alpha=.5)

#plt.xlim(0,np.pi/4)

In [ ]:
# just zooming in on the signalt to examine the noisy bits
plt.plot(t,fx_noisy,'r.',alpha=0.5)
plt.plot(t,fx,'b-',alpha=.5)
plt.xlim(0,np.pi/4)

In [ ]:
# we're calculating a mean value, though since there's a constantly growing 
# component it's not meaningful (deliberately)
fx_noisy.mean()

In [ ]:
# calculate frequencies for bins
# (this uses the scipy fftfreq tool)
frequencies = scipy.fftpack.fftfreq(N_samples,t_stop/N_samples)

frequencies[N_samples//2-1]

In [ ]:
# FFT of our functions
# FFT returns N_samples, but only the first half of the samples are worth looking at (real freqs.)
yf = scipy.fftpack.fft(fx)
yf_noisy = scipy.fftpack.fft(fx_noisy)

# we only need half of the number of samples
xf = np.linspace(0.0, 1.0/(2.0*dt), N_samples//2)

# plot only first half of the transformed array
# plotting only absolute magnitudes (this could be a mistake, as we'll see later)
#plt.plot(xf, 2.0/N_samples * np.abs(yf[0:N_samples//2]),'b-')
#plt.plot(xf, 2.0/N_samples * np.abs(yf_noisy[0:N_samples//2]),'r--')
#plt.grid()
#plt.xlim(-5,30)
#plt.ylim(-1,5)

# plot half of the array; just looking at some of this stuff closely.
plt.plot(frequencies[0:N_samples//2], 2.0/N_samples * np.abs(yf[0:N_samples//2]),'b-')
plt.plot(frequencies[0:N_samples//2], 2.0/N_samples * np.abs(yf_noisy[0:N_samples//2]),'r--')
plt.grid()
plt.xlim(-5,30)




In [ ]:
# now make a similar plot, but with positive and negative - this gives 
# some clues as to which way the amplitude goes!

plt.plot(frequencies,2.0/N_samples*yf_noisy)
plt.grid()
plt.xlim(0,40)
plt.ylim(-2,2)

The cell below this uses the fact that we know where the frequencies are (the `testfreq` list) in an attempt to figure out the magnitude and phase of the sin/cos that make up the signals.  Note that technically everything can be a sin wave with a phase shift.

Also make a test function that is made up of those tested frequencies.  It's not going to be perfect but it's not that far off.

Note that this is a really useful Stack Overflow thing about getting the amplitude and phase: https://dsp.stackexchange.com/questions/72005/calculate-the-magnitude-and-phase-of-a-signal-at-a-particular-frequency-in-pytho 

In [ ]:
# frequencies that I guessed
testfreq=[5.0,11.0,31.0]

test_function = np.zeros_like(t)

for i in testfreq:
    # figure out array index corresponding to this frequency
    index, = np.where(np.isclose(frequencies, i, atol = 1.0/(2.0*dt)/N_samples))

    print(index[0])

    # what's the magnitude of the signal here (correctly normalized by samples)?
    magnitude = np.abs(yf_noisy[index[0]])*2.0/N_samples
    
    # what's the phase of the signal here?
    phase = np.angle(yf_noisy[index[0]])

    print(i,magnitude,phase)
    
    # add to the test function
    test_function += magnitude*np.cos(float(i)*2.0*np.pi*t - phase)

# correct test function to include mean (though this is not super-meaningful)
test_function += fx_noisy.mean()

In [ ]:
# plotting out the whole thing, just for fun.

plt.plot(2.0/N_samples*yf_noisy)
#plt.grid()
#plt.xlim(9500,10000)
#plt.ylim(-2,2)

In [ ]:
# this uses the actual coefficients that I originally put in, normalized by the mean() 
# to get a function.
fx_test_fctn =  -1.5*np.cos(10.0*np.pi*t) \
     + 3.0*np.cos(22.0*np.pi*t) - 1.0*np.sin(62.0*np.pi*t) + fx_noisy.mean()

In [ ]:
# compare the generated signal to the "real" one that doesn't have the constantly-growing
# component in it.  There are some phase shifts, huh?  It's close but not perfect.

#plt.plot(t,fx_noisy,'r.',alpha=0.5)
plt.plot(t,fx,'k-',alpha=.9)
plt.plot(t,fx_test_fctn,'b-',alpha=.5)
plt.plot(t,test_function,'g-',alpha=0.9)
plt.xlim(5,5.5)

Here's where I dirty up the data a bit more.  I'm going to mock this up as an instrument that's having problems, so it "errors out" at times.  I've added two intervals where the data is zeroed out, and there's an error flag.  Both are plotted to make sure it looks OK.

Note that the time intervals where data is available are VERY long, so recovering the whole sample is super-easy.

In [ ]:
# make my error flag array
error_flag = np.zeros_like(t,dtype='int64')

fx_noisy_with_zeros = np.copy(fx_noisy)

# bad interval #1
fx_noisy_with_zeros[np.logical_and(t > 1.5*np.pi, t < 1.75*np.pi)] = 0.0
error_flag[np.logical_and(t > 1.5*np.pi, t < 1.75*np.pi)] = 1

# bad interval #2
fx_noisy_with_zeros[np.logical_and(t > 3.5*np.pi, t < 4.5*np.pi)] = 0.0
error_flag[np.logical_and(t > 3.5*np.pi, t < 4.5*np.pi)] = 1

# what's this look like?
plt.plot(t,fx_noisy_with_zeros)
plt.plot(t,error_flag,'r-')

Use https://numpy.org/doc/stable/reference/generated/numpy.savetxt.html to save the text file.  The np.transpose() is to make everything in columns.

In [ ]:
np.savetxt("instrument_data_output.dat",np.transpose([t,fx_noisy_with_zeros,error_flag]), 
            fmt='%.8f    %.8f    %d',newline='\n',
            header="column 0: time (arb. units)\ncolumn 1: signal amplitude (arb. units)\ncolumn 3: error flag (1=error)\n\nNote: columns are separated by 4 spaces.\n")

